In [39]:
import json
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from sklearn.metrics import classification_report
import wandb
from transformers import BertPreTrainedModel, BertModel, BertConfig
import pickle
from torch.nn.functional import leaky_relu
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [40]:
# Source: https://github.com/LCS2-IIITD/Emotion-Flip-Reasoning/blob/main/Dataloaders/nlp_utils.py
import string
import nltk
import re

numbers = {
    "0":"zero",
    "1":"one",
    "2":"two",
    "3":"three",
    "4":"four",
    "5":"five",
    "6":"six",
    "7":"seven",
    "8":"eight",
    "9":"nine"
}

def remove_puntuations(txt):
    punct = set(string.punctuation)
    txt = " ".join(txt.split("."))
    txt = " ".join(txt.split("!"))
    txt = " ".join(txt.split("?"))
    txt = " ".join(txt.split(":"))
    txt = " ".join(txt.split(";"))
    
    txt = "".join(ch for ch in txt if ch not in punct)
    return txt

def number_to_words(txt):
    for k in numbers.keys():
        txt = txt.replace(k,numbers[k]+" ")
    return txt

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'_',' ',text)
    text = number_to_words(text)
    text = remove_puntuations(text)
    text = ''.join([i if ord(i) < 128 else '' for i in text])
    text = ' '.join(text.split())
    return text

In [41]:
train_data = json.load(open('/kaggle/input/nlp-project/Dataset/ERC_utterance_level/train_utterance_level.json'))
val_data = json.load(open('/kaggle/input/nlp-project/Dataset/ERC_utterance_level/val_utterance_level.json'))

In [42]:
original_train_data = json.load(open('/kaggle/input/nlp-project/Dataset/Original_Dataset/Subtask_1_train.json'))

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [44]:
emotion2int = {
    'anger': 0,
    'joy': 1,
    'fear': 2,
    'disgust': 3,
    'neutral': 4,
    'surprise': 5,
    'sadness': 6
}

In [45]:
L = []
for conversation in original_train_data:
    for utterance in conversation['conversation']:
        L.append(emotion2int[utterance['emotion']])
class_weights=compute_class_weight(class_weight='balanced',classes=list(emotion2int.values()), y=np.array(L))

In [46]:
utterance2vec = pickle.load(open('/kaggle/input/nlp-project/Dataset/Embeddings/sentence_transformer_utterance2vec_384.pkl', 'rb'))

In [47]:
MAX_CONV_LEN = 35
# Defined index 7 for padding
class ERC_Dataset_Utt_Level(Dataset):
    def __init__(self, data, utterance2vec, device):
        self.data = data
        self.utterance2vec = utterance2vec
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[f'id_{idx+1}']['text']
        emotion = self.data[f'id_{idx+1}']['emotion']
        context = self.data[f'id_{idx+1}']['context']

        context_embeddings = [torch.tensor(self.utterance2vec[preprocess_text(utterance)]).to(self.device) for utterance in context]
        target_embedding = torch.tensor(self.utterance2vec[preprocess_text(text)]).to(self.device)
        context_embeddings_cat = [torch.cat((emb, target_embedding)) for emb in context_embeddings]

        if(len(context_embeddings_cat)<MAX_CONV_LEN):
            num_pads = MAX_CONV_LEN - len(context_embeddings_cat)
            attention_mask = [1]*len(context_embeddings_cat) + [0]*num_pads            
            context_embeddings_cat = context_embeddings_cat + [torch.zeros(768).to(self.device)]*num_pads  
            context_embeddings = context_embeddings + [torch.zeros(384).to(self.device)]*num_pads
        else:
            context_embeddings_cat = context_embeddings_cat[len(context_embeddings_cat)-MAX_CONV_LEN:]
            context_embeddings = context_embeddings[len(context_embeddings)-MAX_CONV_LEN:]
            attention_mask = [1]*MAX_CONV_LEN

        context_embeddings_cat = torch.stack(context_embeddings_cat)
        context_embeddings = torch.stack(context_embeddings)
        attention_mask = torch.tensor(attention_mask)
        target_embedding = torch.cat((target_embedding, target_embedding))

        return {
            'context_embeddings_cat': context_embeddings_cat,
            'context_embeddings': context_embeddings,
            'target_embedding': target_embedding,
            'attention_mask': attention_mask,   
            'emotion': emotion2int[emotion]
        }

In [48]:
class BertForSentenceClassificationGivenContext(BertPreTrainedModel):
    def __init__(self, config, weights):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.weights = weights
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size*2, config.num_labels)

        self.post_init()

    def forward(self, context_embeds, target_embeds, attention_mask, labels=None):
        output = self.bert(inputs_embeds=context_embeds, attention_mask=attention_mask)
#         pooled_output = output.pooler_output
#         pooled_output = self.dropout(pooled_output)
#         pooled_output_cat = torch.cat((pooled_output, target_embeds), 1)
#         logits = self.classifier(pooled_output_cat)
        out_target = [output.last_hidden_state[b][int(sum(attention_mask[b]))-1] for b in range(output.last_hidden_state.shape[0])]
        out_target = torch.stack(out_target)
        out_target = self.dropout(out_target)
        out_target_cat = torch.cat((out_target, target_embeds), 1)
        logits = self.classifier(out_target_cat)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.weights.to(device))
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1).to(device))

        return {'loss': loss, 'logits': logits}
        

In [49]:
train_dataset = ERC_Dataset_Utt_Level(train_data, utterance2vec, device)
val_dataset = ERC_Dataset_Utt_Level(val_data, utterance2vec, device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [50]:
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=7)
model = BertForSentenceClassificationGivenContext.from_pretrained('bert-base-uncased', config=config, weights=torch.from_numpy(class_weights).float()).to(device)

Some weights of BertForSentenceClassificationGivenContext were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
epochs = 30
optimizer = AdamW(model.parameters(), lr=1e-6)

In [52]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_login_key")
wandb.login(key=secret_value_0)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [53]:
wandb.init(project='TECPEC', name='BERT_with_target_cat_Utt_Level', config={
    'Embedding': 'Sentence-Transformer',
    'Level': 'Utterance Level',
    'Epochs': epochs,
    'Optimizer': 'AdamW',
    'Learning Rate': 1e-6,
    'Batch Size': 16
})

In [54]:
for epoch in range(epochs):
    model.train()
    train_pred, train_true, train_loss = [], [], 0.0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        context_embeddings_cat, target_embedding, emotions, attention_mask = batch['context_embeddings_cat'].to(device), batch['target_embedding'].to(device), batch['emotion'].to(device), batch['attention_mask'].to(device)
        outputs = model(context_embeds=context_embeddings_cat, target_embeds=target_embedding, attention_mask=attention_mask, labels=emotions)
        loss = outputs['loss']
        loss.backward()
        optimizer.step()
        train_pred.extend(torch.argmax(outputs['logits'], 1).tolist())
        train_true.extend(emotions.tolist())
        train_loss += loss.item()
    train_loss /= len(train_loader) 
    model.eval()
    val_pred, val_true, val_loss = [], [], 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader):
            context_embeddings_cat, target_embedding, emotions, attention_mask = batch['context_embeddings_cat'].to(device), batch['target_embedding'].to(device), batch['emotion'].to(device), batch['attention_mask'].to(device)
            outputs = model(context_embeds=context_embeddings_cat, target_embeds=target_embedding, attention_mask=attention_mask, labels=emotions)
            loss = outputs['loss']
            val_pred.extend(torch.argmax(outputs['logits'], 1).tolist())
            val_true.extend(emotions.tolist())
            val_loss += loss.item()
            
    val_loss /= len(val_loader)
    train_report = classification_report(train_true, train_pred, target_names=emotion2int.keys(), zero_division=0)
    val_report = classification_report(val_true, val_pred, target_names=emotion2int.keys(), zero_division=0)

    train_report_dict = classification_report(train_true, train_pred, target_names=emotion2int.keys(), output_dict=True, zero_division=0)
    val_report_dict = classification_report(val_true, val_pred, target_names=emotion2int.keys(), output_dict=True, zero_division=0)
    wandb.log({
        'train_loss': train_loss,
        'val_loss': val_loss,
        'train_accuracy': train_report_dict['accuracy'],
        'val_accuracy': val_report_dict['accuracy'],
        'Macro train_f1': train_report_dict['macro avg']['f1-score'],
        'Macro val_f1': val_report_dict['macro avg']['f1-score'],
        'Weighted train_f1': train_report_dict['weighted avg']['f1-score'],
        'Weighted val_f1': val_report_dict['weighted avg']['f1-score'],
    })
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss}, Val Loss: {val_loss}")
    print(f"Train Report: \n{train_report}")
    print(f"Val Report: \n{val_report}")


100%|██████████| 93/93 [00:04<00:00, 22.51it/s]


Epoch: 1, Train Loss: 1.957941618204431, Val Loss: 1.9348950578320412
Train Report: 
              precision    recall  f1-score   support

       anger       0.13      0.13      0.13      1423
         joy       0.17      0.20      0.18      2047
        fear       0.02      0.03      0.03       336
     disgust       0.03      0.14      0.05       372
     neutral       0.43      0.21      0.28      5299
    surprise       0.14      0.16      0.15      1656
     sadness       0.10      0.17      0.13      1011

    accuracy                           0.18     12144
   macro avg       0.15      0.15      0.14     12144
weighted avg       0.26      0.18      0.20     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.15      0.17      0.16       192
         joy       0.21      0.27      0.23       254
        fear       0.00      0.00      0.00        37
     disgust       0.05      0.10      0.07        42
     neutral       0.41      0.21 

100%|██████████| 93/93 [00:04<00:00, 22.36it/s]


Epoch: 2, Train Loss: 1.937777233375077, Val Loss: 1.9203401675788305
Train Report: 
              precision    recall  f1-score   support

       anger       0.14      0.16      0.15      1423
         joy       0.20      0.27      0.23      2047
        fear       0.04      0.04      0.04       336
     disgust       0.04      0.06      0.04       372
     neutral       0.46      0.26      0.33      5299
    surprise       0.17      0.23      0.20      1656
     sadness       0.11      0.16      0.13      1011

    accuracy                           0.23     12144
   macro avg       0.17      0.17      0.16     12144
weighted avg       0.29      0.23      0.24     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.18      0.16      0.17       192
         joy       0.25      0.29      0.27       254
        fear       0.06      0.03      0.04        37
     disgust       0.04      0.10      0.06        42
     neutral       0.46      0.27 

100%|██████████| 93/93 [00:04<00:00, 22.44it/s]


...

Epoch: 28, Train Loss: 1.2434481361952068, Val Loss: 1.9963045639376487
Train Report: 
              precision    recall  f1-score   support

       anger       0.39      0.45      0.42      1423
         joy       0.38      0.52      0.44      2047
        fear       0.31      0.76      0.44       336
     disgust       0.29      0.72      0.42       372
     neutral       0.64      0.34      0.44      5299
    surprise       0.48      0.51      0.49      1656
     sadness       0.38      0.55      0.45      1011

    accuracy                           0.45     12144
   macro avg       0.41      0.55      0.44     12144
weighted avg       0.51      0.45      0.45     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.21      0.24      0.22       192
         joy       0.39      0.47      0.43       254
        fear       0.04      0.14      0.06        37
     disgust       0.09      0.21      0.12        42
     neutral       0.55      0.3

100%|██████████| 93/93 [00:04<00:00, 22.64it/s]


Epoch: 29, Train Loss: 1.2056947810298055, Val Loss: 2.025422869190093
Train Report: 
              precision    recall  f1-score   support

       anger       0.41      0.47      0.44      1423
         joy       0.41      0.53      0.46      2047
        fear       0.34      0.77      0.47       336
     disgust       0.35      0.75      0.47       372
     neutral       0.65      0.36      0.46      5299
    surprise       0.47      0.52      0.49      1656
     sadness       0.36      0.54      0.43      1011

    accuracy                           0.46     12144
   macro avg       0.43      0.56      0.46     12144
weighted avg       0.51      0.46      0.46     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.21      0.27      0.24       192
         joy       0.38      0.49      0.43       254
        fear       0.05      0.16      0.08        37
     disgust       0.10      0.14      0.12        42
     neutral       0.57      0.29

100%|██████████| 93/93 [00:04<00:00, 22.47it/s]

Epoch: 30, Train Loss: 1.2017905665402042, Val Loss: 2.0461592366618495
Train Report: 
              precision    recall  f1-score   support

       anger       0.40      0.49      0.44      1423
         joy       0.39      0.56      0.46      2047
        fear       0.33      0.77      0.47       336
     disgust       0.33      0.73      0.46       372
     neutral       0.66      0.33      0.44      5299
    surprise       0.49      0.51      0.50      1656
     sadness       0.39      0.55      0.45      1011

    accuracy                           0.46     12144
   macro avg       0.43      0.57      0.46     12144
weighted avg       0.52      0.46      0.46     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.20      0.26      0.23       192
         joy       0.45      0.41      0.43       254
        fear       0.05      0.16      0.08        37
     disgust       0.09      0.14      0.11        42
     neutral       0.55      0.3

In [55]:
wandb.finish()

Macro train_f1,▁▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
Macro val_f1,▁▃▄▅▄▃▆▅▆▆▇▇▆▇▇▆▇▇▇▇▇▇▇▇█▇█▇▇▇
Weighted train_f1,▁▂▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█▇███
Weighted val_f1,▁▃▃▃▃▁▅▃▅▆▇▆▆▇▆▆▇▇▇▇▇▇▆▇▇▇█▇▇█
train_accuracy,▁▂▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█▇███
train_loss,███▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
val_accuracy,▁▃▃▄▃▂▅▄▅▆▇▆▅▆▆▆▇▇▇▇▆▇▆▇▇▇█▇▇▇
val_loss,▅▄▄▃▃▂▂▁▁▁▁▁▁▂▂▃▃▃▃▄▄▄▄▅▅▆▆▇▇█
Macro train_f1,0.45998
Macro val_f1,0.26058
Weighted train_f1,0.4553


In [56]:
torch.save(model, '/kaggle/working/BERT_add_Utt_Level.pth')